## Célula 1: Importações e Configurações Iniciais


In [1]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml
%pip install openpyxl
%pip install xlrd
%pip install yfinance
%pip install XlsxWriter
%pip install numpy

  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------- -------------------------- 3.1/9.4 MB 14.3 MB/s


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp313-cp313-win_amd64.whl.metadata (36 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.1-cp313-cp313-win_amd64.whl (102 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------ --------------------------- 3.9/12.6 MB 20.0 MB/s eta 0:00:01
   ------------------------------------ --- 11.5/12.6 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 22.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached XlsxWriter-3.2.2-py3-none-any.whl.metadata (2.8 kB)
Using cached XlsxWriter-3.2.2-py3-none-any.whl (165 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Célula 1: Importações e Configurações Iniciais
import os
import zipfile
import pandas as pd
import logging
import yfinance as yf
import math
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from datetime import datetime, timedelta

# Configuração de logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Configuração das pastas
DOWNLOADS_FOLDER = "downloads"
BALANCOS_FOLDER = "balancos"
BALANCOS_XLSX_FOLDER = "balancos_xlsx"
BALANCOS_CONCATENADOS_FOLDER = "balancos_concatenados"
BALANCOS_DEFINITIVO_FOLDER = "balancos_definitivos"
COTACOES_FOLDER = "cotacoes"
BALANCOS_COTACOES_COMBINADOS_FOLDER = "balancos_cotacoes_combinados"


# Criar pastas se não existirem
for folder in [
    DOWNLOADS_FOLDER,
    BALANCOS_FOLDER,
    BALANCOS_XLSX_FOLDER,
    BALANCOS_CONCATENADOS_FOLDER,
    BALANCOS_DEFINITIVO_FOLDER,
    COTACOES_FOLDER,
    BALANCOS_COTACOES_COMBINADOS_FOLDER,
]:
    os.makedirs(folder, exist_ok=True)

## Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados


In [3]:
# Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados
# Configuração do WebDriver em modo headless
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# URL da página de resultados
url_resultado = "https://www.fundamentus.com.br/resultado.php"


# Função para obter a tabela HTML
def obter_tabela_html(url, xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elemento = driver.find_element("xpath", xpath)
    return elemento.get_attribute("outerHTML")


# Obter a tabela HTML
html_tabela = obter_tabela_html(url_resultado, "/html/body/div[1]/div[2]/table")

# Converter a tabela HTML em DataFrame
df_dados_financeiros = pd.read_html(StringIO(html_tabela), decimal=",", thousands=".")[
    0
]
df_dados_financeiros = df_dados_financeiros.sort_values(
    by="Papel", ascending=True
).reset_index(drop=True)

# Exibir o DataFrame
print("Tabela com os dados financeiros:")
display(df_dados_financeiros)

# Lista de empresas
lista_de_empresas = df_dados_financeiros["Papel"].tolist()
print("Quantidade de empresas:", len(lista_de_empresas))

# Fechar o WebDriver
driver.quit()

Tabela com os dados financeiros:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,AALR3,6.85,-8.08,0.95,0.858,"0,00%",0.361,37.23,9.75,-0.99,...,7.68,"8,80%","-9,63%",1.04,"3,97%","-11,79%",3330440.0,1.094790e+09,0.66,"5,95%"
1,ABCB3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","15,21%",0.0,6.387350e+09,0.00,"74,29%"
2,ABCB4,20.14,5.07,0.77,0.000,"8,17%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","15,21%",12592900.0,6.387350e+09,0.00,"74,29%"
3,ABEV3,13.61,14.85,2.17,2.397,"5,84%",1.320,44.99,11.03,-24.45,...,7.18,"21,74%","16,60%",1.10,"17,90%","14,64%",424894000.0,9.864210e+10,0.04,"9,90%"
4,ABYA3,4.91,-214.80,1.76,2.055,"0,00%",0.527,1.98,19.96,-2.75,...,33.67,"10,29%","-0,96%",2.09,"2,78%","-0,82%",0.0,2.920600e+08,1.31,"16,41%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,WLMM4,30.89,8.69,1.45,0.360,"4,49%",0.857,2.38,6.23,4.14,...,7.56,"5,78%","4,14%",2.40,"15,11%","16,73%",29371.8,7.732220e+08,0.48,"28,95%"
987,WMBY3,25.39,-19.30,2.87,0.836,"0,00%",0.182,1.20,8.62,-1.50,...,25.02,"9,70%","-7,05%",1.44,"2,39%","-14,86%",0.0,2.124390e+08,6.51,"-14,48%"
988,WSON33,67.00,8.07,0.98,1.067,"0,00%",0.400,13.41,2.42,-0.89,...,5.08,"44,14%","13,58%",1.26,"18,36%","12,17%",0.0,2.148530e+09,1.21,"5,23%"
989,YDUQ3,12.04,9.67,1.06,0.617,"2,56%",0.332,3.05,3.80,-0.77,...,4.64,"16,21%","6,38%",1.75,"10,05%","10,92%",58063400.0,3.124960e+09,1.79,"8,50%"


Quantidade de empresas: 991


In [4]:
df_dados_financeiros.to_excel("dados_financeiros.xlsx", index=False)

## Célula 3: Funções Auxiliares


In [5]:
# Célula 3: Funções Auxiliares
def obter_trimestre_anterior():
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    if mes_atual <= 3:
        return f"Q4-{ano_atual-1}"
    elif mes_atual <= 6:
        return f"Q1-{ano_atual}"
    elif mes_atual <= 9:
        return f"Q2-{ano_atual}"
    else:
        return f"Q3-{ano_atual}"


trimestre_anterior = obter_trimestre_anterior()
pasta_trimestre = os.path.join(DOWNLOADS_FOLDER, trimestre_anterior)
os.makedirs(pasta_trimestre, exist_ok=True)

## Célula 4: Download dos Arquivos


### Download dos balanços


In [ ]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), pasta_trimestre),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)


def baixar_e_renomear_arquivo(papel):
    arquivo_destino = os.path.join(pasta_trimestre, f"{papel}.zip")
    if os.path.isfile(arquivo_destino):
        logging.info(f"Arquivo {arquivo_destino} já existe. Pulando o papel {papel}.")
        return

    url_papeis = (
        f"https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile"
    )
    driver.get(url_papeis)

    try:
        time.sleep(2)
        botao_download = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.bt-baixar"))
        )
        botao_download.click()

        # Esperar o download começar
        for _ in range(10):
            time.sleep(5)
            if any(f.startswith("bal_") for f in os.listdir(pasta_trimestre)):
                break

        novo_arquivo = next(
            (f for f in os.listdir(pasta_trimestre) if f.startswith("bal_")), None
        )

        if novo_arquivo:
            caminho_arquivo = os.path.join(pasta_trimestre, novo_arquivo)
            os.rename(caminho_arquivo, arquivo_destino)
            logging.info(f"Arquivo {arquivo_destino} baixado e renomeado com sucesso!")
        else:
            logging.error(f"Erro ao encontrar o arquivo baixado para {papel}")

    except Exception as e:
        logging.error(f"Erro ao baixar arquivo para {papel}: {e}")


# Baixar e renomear arquivos para cada empresa
for papel in lista_de_empresas:
    baixar_e_renomear_arquivo(papel)

# Fechar o WebDriver
driver.quit()

### Obter Cotações e Salvar em Excel


In [ ]:
tickers_falhados = []
contador_de_empresas = 0

# Itera sobre a lista de empresas
for papel in lista_de_empresas:
    # Adiciona ".SA" ao nome do papel para buscar na B3
    papel_com_sufixo = papel + ".SA"

    # Cria o caminho do arquivo Excel
    arquivo_excel = os.path.join(COTACOES_FOLDER, f"{papel}.xlsx")

    # Verifica se o arquivo já foi baixado
    if os.path.exists(arquivo_excel):
        print(
            f"As cotações para {papel_com_sufixo} já foram baixadas anteriormente. Pulando..."
        )
        contador_de_empresas += 1  # Contabiliza como sucesso, já que já existe
        continue

    try:
        print(f"Obtendo cotações para {papel_com_sufixo}...")

        # Obtém os dados de mercado usando o Yahoo Finance
        cotacao = yf.download(papel_com_sufixo, period="max")

        # Verifica se os dados foram obtidos corretamente
        if not cotacao.empty:
            # Remove as horas da coluna 'Date' e formata para 'dd/mm/yyyy'
            cotacao.index = pd.to_datetime(cotacao.index).strftime("%d/%m/%Y")

            # Transpõe o DataFrame para ter as datas como colunas
            cotacao_transposta = cotacao.transpose()

            # Escreve no Excel
            with pd.ExcelWriter(arquivo_excel, engine="xlsxwriter") as writer:
                cotacao_transposta.to_excel(writer, sheet_name="Cotações")

                # Obtém o workbook e worksheet
                workbook = writer.book
                worksheet = writer.sheets["Cotações"]

                # Formata a primeira linha (as datas) no formato dd/mm/yyyy
                date_format = workbook.add_format({"num_format": "dd/mm/yyyy"})
                worksheet.set_row(0, None, date_format)

            print(
                f"Cotações de {papel_com_sufixo} salvas com sucesso em {arquivo_excel}"
            )
            contador_de_empresas += 1
        else:
            print(f"Cotações não encontradas para {papel_com_sufixo}")
            tickers_falhados.append(papel_com_sufixo)

    except Exception as e:
        print(f"Erro ao baixar cotações para {papel_com_sufixo}: {e}")
        tickers_falhados.append(papel_com_sufixo)

# Exibe o resultado
print(f"\nTotal de empresas com cotações obtidas com sucesso: {contador_de_empresas}")
if tickers_falhados:
    print(f"Empresas para as quais as cotações falharam: {', '.join(tickers_falhados)}")

## Célula 5: Extração e Renomeação dos Arquivos


In [ ]:
# Célula 5: Extração e Renomeação dos Arquivos

# Configuração básica do logging
logging.basicConfig(filename="extracao_renomeacao.log", level=logging.ERROR)


# Função de extração e renomeação
def extrair_e_renomear_arquivo(papel):
    caminho_arquivo_zip = os.path.join(pasta_trimestre, f"{papel}.zip")
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_trimestre, exist_ok=True)
    caminho_arquivo_xls = os.path.join(pasta_balancos_trimestre, "balanco.xls")
    novo_nome_arquivo = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")

    # Verifica se o arquivo já foi extraído e renomeado
    if os.path.exists(novo_nome_arquivo):
        print(f"Arquivo {novo_nome_arquivo} já foi extraído e renomeado.")
        return True  # Indica sucesso sem necessidade de renomeação

    try:
        # Extraindo o arquivo ZIP
        with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
            zip_ref.extractall(pasta_balancos_trimestre)

        # Renomeando o arquivo
        if os.path.exists(caminho_arquivo_xls):
            os.rename(caminho_arquivo_xls, novo_nome_arquivo)
            print(f"Arquivo {novo_nome_arquivo} extraído e renomeado com sucesso!")
        else:
            print(f"Arquivo balanco.xls não encontrado em {pasta_balancos_trimestre}")
            return False  # Indica falha na renomeação

    except Exception as e:
        logging.error(f"Erro ao extrair e renomear arquivo para {papel}: {e}")
        return False  # Indica falha na operação

    return True  # Indica sucesso


# Lista para armazenar papéis com erro
papeis_com_erro = []

# Extração e renomeação dos arquivos para cada empresa
for papel in lista_de_empresas[
    :
]:  # Usar uma cópia da lista para evitar problemas ao remover elementos
    sucesso = extrair_e_renomear_arquivo(papel)
    if not sucesso:
        papeis_com_erro.append(papel)

# Remover os papéis com erro da lista de empresas
for papel in papeis_com_erro:
    lista_de_empresas.remove(papel)

# Exibir os papéis que deram erro
print("PAPÉIS COM ERRO:", papeis_com_erro)
print("LISTA DE EMPRESAS ATUALIZADA:", lista_de_empresas)
print("QUANTIDADE DE EMPRESAS ATUALIZADA:", len(lista_de_empresas))

## Célula 6: Conversão de XLS para XLSX


In [ ]:
# Célula 6: Conversão de XLS para XLSX
def converter_xls_para_xlsx(papel):
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    pasta_balancos_xlsx_trimestre = os.path.join(
        BALANCOS_XLSX_FOLDER, trimestre_anterior
    )
    os.makedirs(pasta_balancos_xlsx_trimestre, exist_ok=True)

    caminho_xls = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")
    caminho_xlsx = os.path.join(pasta_balancos_xlsx_trimestre, f"{papel}.xlsx")

    if os.path.isfile(caminho_xlsx):
        print(f"Arquivo {papel}.xlsx já foi convertido.")
        return

    try:
        xls = pd.ExcelFile(caminho_xls, engine="xlrd")
        with pd.ExcelWriter(caminho_xlsx, engine="openpyxl") as writer:
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Arquivo {papel}.xls convertido para {papel}.xlsx.")
    except Exception as e:
        print(f"Erro ao converter {papel}.xls: {e}")


# Converter arquivos XLS para XLSX para cada empresa
for papel in lista_de_empresas:
    converter_xls_para_xlsx(papel)

## Concatenação das Tabelas


In [ ]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Constantes
BAL_PATRIM = "Bal. Patrim."
DEM_RESULT = "Dem. Result."
EXTENSAO_EXCEL = ".xlsx"


def reordenar_primeira_linha(df: pd.DataFrame) -> pd.DataFrame:
    """
    Reordena a primeira linha de um DataFrame, colocando as datas da mais antiga para a mais recente.
    """
    # Identificar a primeira linha (após remoção da original) para as datas
    primeira_linha = df.iloc[0, 1:]

    # Tentar converter os valores para datetime, forçando o formato "dd/mm/yyyy"
    datas = pd.to_datetime(primeira_linha, format="%d/%m/%Y", errors="coerce")

    # Verificar se todas as datas são válidas
    if datas.isnull().all():
        logging.warning(
            "Nenhuma data foi reconhecida na primeira linha. Retornando DataFrame original."
        )
        return df

    # Criar um DataFrame temporário para facilitar a reordenação
    temp_df = pd.DataFrame({"original_col": df.columns[1:], "date": datas})

    # Remover entradas que não são datas válidas
    temp_df = temp_df.dropna(subset=["date"])

    # Ordenar as colunas com base nas datas
    temp_df = temp_df.sort_values(by="date")

    # Reordenar o DataFrame original de acordo com as colunas ordenadas
    new_order = [df.columns[0]] + temp_df["original_col"].tolist()
    df = df[new_order]

    # Substituir a primeira linha com as datas ordenadas
    df.iloc[0, 1:] = temp_df["date"].dt.strftime("%d/%m/%Y").values

    return df


def processar_arquivo(caminho_arquivo: str, pasta_saida: str) -> None:
    try:
        nome_arquivo = os.path.basename(caminho_arquivo)
        nome_papel = nome_arquivo.replace(EXTENSAO_EXCEL, "")

        bal_patrim = pd.read_excel(
            caminho_arquivo, sheet_name=BAL_PATRIM, header=None, engine="openpyxl"
        )
        dem_result = pd.read_excel(
            caminho_arquivo, sheet_name=DEM_RESULT, header=None, engine="openpyxl"
        )

        # Remover a primeira linha de ambas as planilhas
        bal_patrim = bal_patrim.iloc[1:].reset_index(drop=True)
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Reordenar a primeira linha de ambas as planilhas
        bal_patrim = reordenar_primeira_linha(bal_patrim)
        dem_result = reordenar_primeira_linha(dem_result)

        # Verificar o conteúdo da célula [0, 0] da planilha 'Bal. Patrim.'
        primeira_celula = bal_patrim.iloc[0, 0]

        if (
            pd.to_datetime(primeira_celula, format="%d/%m/%Y", errors="coerce")
            is not pd.NaT
        ):
            # Se a célula contém uma data, mover as colunas para a direita
            bal_patrim = bal_patrim.shift(1, axis=1)

        # Definir o nome do papel na célula [0, 0] da planilha 'Bal. Patrim.'
        bal_patrim.iloc[0, 0] = nome_papel

        # Remover a primeira linha de 'Dem. Result.' antes de concatenar
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Concatenar as duas planilhas
        combinado = pd.concat([bal_patrim, dem_result], ignore_index=True)

        # Verificação final: Se a célula [0, 0] não contiver o nome do papel, remover a primeira linha
        if combinado.iloc[0, 0] != nome_papel:
            combinado = combinado.iloc[1:].reset_index(drop=True)

        # Criar a pasta de saída correspondente
        nome_subpasta = os.path.basename(os.path.dirname(caminho_arquivo))
        subpasta_saida = os.path.join(pasta_saida, nome_subpasta)
        os.makedirs(subpasta_saida, exist_ok=True)

        # Criar o caminho para o arquivo de saída
        arquivo_saida = os.path.join(subpasta_saida, nome_arquivo)

        # Verificar se o arquivo já existe antes de salvar
        if os.path.exists(arquivo_saida):
            logging.info(f"Arquivo já existe e será ignorado: {arquivo_saida}")
        else:
            # Salvar o resultado em uma nova planilha
            combinado.to_excel(arquivo_saida, index=False)
            logging.info(f"Arquivo combinado e ordenado salvo como: {arquivo_saida}")

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo {caminho_arquivo}: {str(e)}")


def processar_todos_arquivos(pasta_base: str, pasta_saida: str) -> None:
    for nome_subpasta in sorted(os.listdir(pasta_base)):
        caminho_subpasta = os.path.join(pasta_base, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            for nome_arquivo in sorted(os.listdir(caminho_subpasta)):
                if nome_arquivo.endswith(EXTENSAO_EXCEL):
                    caminho_arquivo = os.path.join(caminho_subpasta, nome_arquivo)
                    processar_arquivo(caminho_arquivo, pasta_saida)


# Execução do processamento
processar_todos_arquivos(BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER)

In [ ]:
# Função para obter as subpastas ordenadas da mais antiga para a mais recente
def obter_subpastas_ordenadas(base_folder):
    subpastas = [f.name for f in os.scandir(base_folder) if f.is_dir()]
    subpastas.sort(
        key=lambda x: datetime.strptime(x.split("-")[1], "%Y")
    )  # Ordenar por ano
    return subpastas


# Função para concatenar as planilhas
def concatenar_planilhas(base_folder, output_folder):
    planilhas_dict = {}

    # Obter subpastas ordenadas
    subpastas_ordenadas = obter_subpastas_ordenadas(base_folder)

    # Percorre todas as subpastas (ordenadas pela data da pasta mais antiga para a mais recente)
    for subpasta in subpastas_ordenadas:
        subpasta_path = os.path.join(base_folder, subpasta)
        if os.path.isdir(subpasta_path):
            for file_name in os.listdir(subpasta_path):
                if file_name.endswith(".xlsx"):
                    file_path = os.path.join(subpasta_path, file_name)

                    # Carregar a planilha, ignorando a primeira linha (índice de colunas)
                    df = pd.read_excel(file_path, skiprows=1)

                    # Se o arquivo já foi processado, concatenar as novas colunas
                    if file_name in planilhas_dict:
                        df_existente = planilhas_dict[file_name]

                        # Ignorar a primeira coluna (indicadores financeiros)
                        for col in df.columns[1:]:
                            if col not in df_existente.columns:
                                df_existente[col] = df[col]

                        # Reordenar as colunas pela data
                        colunas_datas = [
                            col
                            for col in df_existente.columns
                            if col != df_existente.columns[0]
                        ]
                        colunas_datas.sort(
                            key=lambda date: datetime.strptime(str(date), "%d/%m/%Y")
                        )
                        colunas_ordenadas = [df_existente.columns[0]] + colunas_datas
                        planilhas_dict[file_name] = df_existente[colunas_ordenadas]
                    else:
                        # Armazenar a primeira planilha encontrada
                        planilhas_dict[file_name] = df

    # Salvar os arquivos concatenados na pasta de destino
    for nome_arquivo, df_final in planilhas_dict.items():
        # Salvar o DataFrame concatenado no arquivo de destino
        output_path = os.path.join(output_folder, nome_arquivo)
        df_final.to_excel(output_path, index=False)


# Executar a função
concatenar_planilhas(BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER)

## Dicionarios


### Dicionario de balanços


In [4]:
# Dicionário para armazenar os DataFrames
dicionario_de_balancos = {}
contador_de_empresas = 0
# Percorre todos os arquivos na pasta 'balancos_definitivos'
for arquivo in os.listdir(BALANCOS_DEFINITIVO_FOLDER):
    if arquivo.endswith(".xlsx"):
        # Cria o caminho completo para o arquivo
        caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)
        contador_de_empresas = contador_de_empresas + 1
        # Carrega o DataFrame a partir do arquivo Excel
        balanco = pd.read_excel(
            caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
        )

        # Na célula [0, 0] está o nome da empresa
        nome = balanco.iloc[0, 0]
        print(f"Processando empresa {nome}...")
        # Pega a 1ª linha e torna um cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]

        # Define a 1ª coluna (nome da empresa) como índice
        balanco = balanco.set_index(nome)

        # Adiciona o DataFrame ao dicionário com o nome da empresa como chave
        dicionario_de_balancos[nome] = balanco
print("TOTAL DE EMPRESAS: ", contador_de_empresas)

Processando empresa AALR3...
Processando empresa ABCB3...
Processando empresa ABCB4...
Processando empresa ABEV3...
Processando empresa ABYA3...
Processando empresa ACES3...
Processando empresa ACES4...
Processando empresa ADHM3...
Processando empresa AEDU11...
Processando empresa AEDU3...
Processando empresa AELP3...
Processando empresa AERI3...
Processando empresa AESB3...
Processando empresa AESL3...
Processando empresa AESL4...
Processando empresa AFLT3...
Processando empresa AFLU3...
Processando empresa AFLU5...
Processando empresa AGEI3...
Processando empresa AGEN33...
Processando empresa AGIN3...
Processando empresa AGRO3...
Processando empresa AGXY3...
Processando empresa AHEB3...
Processando empresa AHEB5...
Processando empresa AHEB6...
Processando empresa ALBA3...
Processando empresa ALLD3...
Processando empresa ALLL11...
Processando empresa ALLL3...
Processando empresa ALLL4...
Processando empresa ALOS3...
Processando empresa ALPA3...
Processando empresa ALPA4...
Processando

In [5]:
# Conjunto de nomes no dicionário
nomes_no_dicionario = set(dicionario_de_balancos.keys())

# Conjunto de nomes na lista de empresas
nomes_na_lista = set(lista_de_empresas)

# Empresas que estão no dicionário mas não na lista de empresas
faltando_na_lista = nomes_no_dicionario - nomes_na_lista

# Atualizar lista_de_empresas para incluir as empresas que estão no dicionário
lista_de_empresas.extend(faltando_na_lista)

# Remover duplicatas da lista_de_empresas (caso haja)
lista_de_empresas = list(set(lista_de_empresas))
sorted_lista_de_empresas = sorted(lista_de_empresas)
print("LISTA ATUALIZADA DE EMPRESAS:")
print(sorted_lista_de_empresas)
print("QUANTIDADE DE EMPRESAS ATUALIZADA:", len(sorted_lista_de_empresas))

# Opcional: Exibir as empresas que estavam no dicionário, mas não na lista antes da atualização
print("Empresas que foram adicionadas à lista:")
print(faltando_na_lista)
print("QUANTIDADE DE EMPRESAS ADICIONADAS:", len(faltando_na_lista))

LISTA ATUALIZADA DE EMPRESAS:
['AALR3', 'ABCB3', 'ABCB4', 'ABEV3', 'ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AERI3', 'AESB3', 'AESL3', 'AESL4', 'AFLT3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALBA3', 'ALLD3', 'ALLL11', 'ALLL3', 'ALLL4', 'ALOS3', 'ALPA3', 'ALPA4', 'ALPK3', 'ALSC3', 'ALSO3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3', 'AMBP3', 'AMBV3', 'AMBV4', 'AMER3', 'AMIL3', 'AMOB3', 'AMPI3', 'ANIM3', 'APER3', 'APTI4', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARML3', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARTR3', 'ARZZ3', 'ASAI3', 'ASSM3', 'ASSM4', 'ASTA4', 'ATED3', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AUTM3', 'AVIL3', 'AVLL3', 'AZEV3', 'AZEV4', 'AZTE3', 'AZUL4', 'AZZA3', 'B3SA3', 'BAHI11', 'BAHI3', 'BAHI4', 'BAHI5', 'BALM3', 'BALM4', 'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBRK3', 'BBSE3', 'BBTG11', 'BBTG12', 'BBTG13', 'BCAL6', 'BDLL3', 'BDLL4', 'BECE3', 'BECE4', 'BEEF3', 'BEES3', 'BEES4', 'BELG3', 'BELG4

In [6]:
# Configurações
data_referencia = "01/01/2023"
data_referencia_aux = pd.to_datetime(data_referencia, format="%d/%m/%Y")

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com último balanço anterior à data de referência
for papel, balanco in dicionario_de_balancos.items():
    colunas = list(balanco.columns)

    try:
        # Converte a última coluna para datetime, especificando o formato e dayfirst=True
        ultima_data_aux = pd.to_datetime(colunas[-1], format="%d/%m/%Y", dayfirst=True)

        if ultima_data_aux <= data_referencia_aux:
            lista_para_remover.append(papel)
    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_balancos.pop(papel)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_balancos.keys()))

LISTA DE EMPRESAS A SEREM REMOVIDAS:  ['ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'ALBA3', 'ALSC3', 'ALSO3', 'AMBV3', 'AMBV4', 'AMIL3', 'AMPI3', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARZZ3', 'ASSM3', 'ASSM4', 'ASTA4', 'AUTM3', 'AVIL3', 'BBRK3', 'BBSE3', 'BBTG11', 'BCAL6', 'BECE3', 'BECE4', 'BELG3', 'BELG4', 'BEMA3', 'BERG3', 'BESP3', 'BESP4', 'BFIT3', 'BFIT4', 'BHGR3', 'BICB3', 'BICB4', 'BIDI11', 'BIDI3', 'BIDI4', 'BISA3', 'BKBR3', 'BLUT3', 'BLUT4', 'BMEF3', 'BMTO3', 'BMTO4', 'BNCA3', 'BOVH3', 'BPAT33', 'BPHA3', 'BPIA3', 'BRDT3', 'BRTP3', 'BRTP4', 'BSCT3', 'BSCT5', 'BSCT6', 'BSEV3', 'BSGR3', 'BTTL3', 'BUET3', 'BUET4', 'CAFE3', 'CAFE4', 'CARD3', 'CCHI3', 'CCHI4', 'CCIM3', 'CCPR3', 'CCTU4', 'CCXC3', 'CELM3', 'CFLU4', 'CGOS3', 'CGOS4', 'CIQU3', 'CIQU4', 'CLAN3', 'CLAN4', 'CMET4', 'CMGI3', 'CMGI4', 'CMMA4', 'CNFB4', 'CPNY3', 'CPSL3', 'CRBM3', 'CRBM7', 'CRDE3', 'CREM3', 'CREM4', 'C

In [7]:
dicionario_de_balancos["PETR4"]

,30/06/2009,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,30/06/2011,30/09/2011,...,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024
PETR4,,,,,,,,,,,,,,,,,,,,,
Ativo Total,305265344.512,333789528.064,345607274.496,365998080,382029856.768,507697037.312,519970029.568,544945274.88,554583457.792,582124896.256,...,972950994.944,998662012.928,1004753977.344,947574013.952,976708960.256,978576998.4,990459002.88,1025495990.272,1050887979.008,1067292033.024
Ativo Circulante,57621536.768,75719057.408,76674015.232,74459103.232,71980236.8,111415033.856,106685161.472,120036376.576,119493476.352,120493400.064,...,168247001.088,199511998.464,224784990.208,159327010.816,163051995.136,157193994.24,136816001.024,147311001.6,157079011.328,165964005.376
Caixa e Equivalentes de Caixa,10072162.304,30088286.208,28795713.536,26951325.696,24209866.752,47291932.672,30323259.392,43344818.176,34672615.424,33659195.392,...,58410000.384,81601003.52,85310996.48,23650000.896,41722998.784,52276998.144,49882001.408,60642000.896,61612998.656,57689001.984
Aplicações Financeiras,0,0,0,0,0,0,26017296.384,20015771.648,24969263.104,21410791.424,...,3630000.128,5967000.064,14956999.68,13038000.128,14469999.616,14629000.192,11102999.552,6504999.936,13649999.872,24071999.488
Contas a Receber,14555268.096,13643311.104,13984270.336,16200354.816,15961581.568,18407325.696,17333975.04,17777672.192,18762930.176,20466372.608,...,35538001.92,26848999.424,26442999.808,22026000.384,26141999.104,23497000.96,21041000.448,25501999.104,29702000.64,25184000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IR Diferido,-679049.024,-1329499.008,446287.008,-1541420.032,-2775495.936,-1923373.952,-2373476.096,-2373476.096,-1750392.064,925491.968,...,-10135000.064,-136000,-1222000,6975000.064,-3497999.872,-4003000.064,1504999.936,1453999.872,1656999.936,5568000
Participações/Contribuições Estatutárias,0,-1495323.008,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reversão dos Juros sobre Capital Próprio,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dicionario de cotações


In [8]:
# Dicionário para armazenar os DataFrames das cotações
dicionario_de_cotacoes = {}
contador_de_empresas = 0

# Percorre todos os arquivos na pasta de cotações
for arquivo in os.listdir(COTACOES_FOLDER):
    if arquivo.endswith(".xlsx"):
        # Cria o caminho completo para o arquivo
        caminho_arquivo = os.path.join(COTACOES_FOLDER, arquivo)

        # O nome do papel é o nome do arquivo sem a extensão
        nome_papel = os.path.splitext(arquivo)[0]
        contador_de_empresas += 1

        # Carrega o DataFrame a partir do arquivo Excel
        cotacao = pd.read_excel(
            caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
        )

        # Configura a célula [0,0] com o nome do papel
        cotacao.iloc[0, 0] = nome_papel
        print(f"Processando empresa {nome_papel}...")
        # Define a primeira linha como os nomes das colunas e a primeira coluna como índice
        cotacao.columns = cotacao.iloc[0]  # Primeira linha vira cabeçalho
        cotacao = cotacao[1:]  # Remove a linha que foi usada como cabeçalho
        cotacao.set_index(
            nome_papel, inplace=True
        )  # Define a coluna do nome do papel como índice

        # Adiciona o DataFrame ao dicionário
        dicionario_de_cotacoes[nome_papel] = cotacao

print("TOTAL DE EMPRESAS: ", contador_de_empresas)

Processando empresa AALR3...
Processando empresa ABCB4...
Processando empresa ABEV3...
Processando empresa ADHM3...
Processando empresa AELP3...
Processando empresa AERI3...
Processando empresa AESB3...
Processando empresa AFLT3...
Processando empresa AGRO3...
Processando empresa AGXY3...
Processando empresa AHEB3...
Processando empresa AHEB5...
Processando empresa AHEB6...
Processando empresa ALLD3...
Processando empresa ALOS3...
Processando empresa ALPA3...
Processando empresa ALPA4...
Processando empresa ALPK3...
Processando empresa ALUP11...
Processando empresa ALUP3...
Processando empresa ALUP4...
Processando empresa AMAR3...
Processando empresa AMBP3...
Processando empresa AMER3...
Processando empresa ANIM3...
Processando empresa APER3...
Processando empresa APTI4...
Processando empresa ARML3...
Processando empresa ARZZ3...
Processando empresa ASAI3...
Processando empresa ATMP3...
Processando empresa ATOM3...
Processando empresa AURA33...
Processando empresa AURE3...
Processando 

In [9]:
# Conjunto de nomes no dicionário de cotações
nomes_no_dicionario_cotacoes = set(dicionario_de_cotacoes.keys())

# Conjunto de nomes na lista de empresas
nomes_na_lista = set(lista_de_empresas)

# Empresas que estão no dicionário de cotações, mas não na lista de empresas
faltando_na_lista = nomes_no_dicionario_cotacoes - nomes_na_lista

# Atualizar lista_de_empresas para incluir as empresas que estão no dicionário
lista_de_empresas.extend(faltando_na_lista)

# Remover duplicatas da lista_de_empresas (caso haja)
lista_de_empresas = list(set(lista_de_empresas))
sorted_lista_de_empresas = sorted(lista_de_empresas)

print("LISTA ATUALIZADA DE EMPRESAS:")
print(sorted_lista_de_empresas)

# Opcional: Exibir as empresas que estavam no dicionário, mas não na lista antes da atualização
print("Empresas que foram adicionadas à lista:")
print(faltando_na_lista)

LISTA ATUALIZADA DE EMPRESAS:
['AALR3', 'ABCB3', 'ABCB4', 'ABEV3', 'ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AERI3', 'AESB3', 'AESL3', 'AESL4', 'AFLT3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALBA3', 'ALLD3', 'ALLL11', 'ALLL3', 'ALLL4', 'ALOS3', 'ALPA3', 'ALPA4', 'ALPK3', 'ALSC3', 'ALSO3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3', 'AMBP3', 'AMBV3', 'AMBV4', 'AMER3', 'AMIL3', 'AMOB3', 'AMPI3', 'ANIM3', 'APER3', 'APTI4', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARML3', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARTR3', 'ARZZ3', 'ASAI3', 'ASSM3', 'ASSM4', 'ASTA4', 'ATED3', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AUTM3', 'AVIL3', 'AVLL3', 'AZEV3', 'AZEV4', 'AZTE3', 'AZUL4', 'AZZA3', 'B3SA3', 'BAHI11', 'BAHI3', 'BAHI4', 'BAHI5', 'BALM3', 'BALM4', 'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBRK3', 'BBSE3', 'BBTG11', 'BBTG12', 'BBTG13', 'BCAL6', 'BDLL3', 'BDLL4', 'BECE3', 'BECE4', 'BEEF3', 'BEES3', 'BEES4', 'BELG3', 'BELG4

In [10]:
# Configurações
data_referencia = "01/01/2023"
data_referencia_aux = pd.to_datetime(data_referencia, format="%d/%m/%Y")

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com última cotação anterior à data de referência
for papel, cotacao in dicionario_de_cotacoes.items():
    try:
        # Obtém a última data do DataFrame de cotações
        ultima_data_aux = pd.to_datetime(
            cotacao.columns[-1], format="%d/%m/%Y", dayfirst=True
        )

        if ultima_data_aux <= data_referencia_aux:
            lista_para_remover.append(papel)
    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS:", lista_para_remover)

# Remove as empresas do dicionário de cotações
for papel in lista_para_remover:
    dicionario_de_cotacoes.pop(papel)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_cotacoes.keys()))

LISTA DE EMPRESAS A SEREM REMOVIDAS: ['AELP3', 'AUTM3', 'BHGR3', 'BUET3', 'CBMA3', 'CBMA4', 'CREM3', 'CZLT33', 'CZRS4', 'DAGB33', 'DHBI3', 'DHBI4', 'ELPL4', 'FBMC3', 'FBMC4', 'FIBR3', 'FTRX4', 'IGUA3', 'IGUA5', 'IMBI3', 'IMBI4', 'KROT11', 'LFFE3', 'LFFE4', 'MTIG3', 'OGXP3', 'PRML3', 'PRTX3', 'PRVI3', 'RDCD3', 'REEM4', 'RJCP3', 'SCLO3', 'SCLO4', 'SJOS3', 'SJOS4', 'SULT3', 'SULT4', 'TERI3', 'TNCP4', 'VALE5', 'VVAR11', 'VVAR4']

TOTAL REMANESCENTE: 475


In [11]:
dicionario_de_cotacoes["PETR4"]

,03/01/2000,04/01/2000,05/01/2000,06/01/2000,07/01/2000,10/01/2000,11/01/2000,12/01/2000,13/01/2000,14/01/2000,...,12/09/2024,13/09/2024,16/09/2024,17/09/2024,18/09/2024,19/09/2024,20/09/2024,23/09/2024,24/09/2024,25/09/2024
PETR4,,,,,,,,,,,,,,,,,,,,,
Open,5.875,5.55,5.494,5.475,5.5,5.613,5.475,5.483,5.325,5.4,...,37.290001,37.110001,37.240002,37.169998,36.91,36.57,36.27,36.389999,37.23,37.099998
High,5.875,5.55,5.494,5.475,5.5,5.613,5.475,5.483,5.325,5.4,...,37.32,37.549999,37.720001,37.189999,36.93,36.779999,36.400002,36.959999,37.310001,37.43
Low,5.875,5.55,5.494,5.475,5.5,5.613,5.475,5.483,5.325,5.4,...,36.75,36.59,37.16,36.700001,36.150002,36.189999,36.07,36.220001,36.720001,36.959999
Close,5.875,5.55,5.494,5.475,5.5,5.613,5.475,5.483,5.325,5.4,...,36.869999,36.700001,37.209999,37.040001,36.150002,36.27,36.259998,36.630001,36.779999,36.990002
Adj Close,1.342284,1.268031,1.255236,1.250895,1.256606,1.282424,1.250895,1.252723,1.216624,1.233759,...,36.869999,36.700001,37.209999,37.040001,36.150002,36.27,36.259998,36.630001,36.779999,36.990002
Volume,35389440000,28861440000,43033600000,34055680000,20912640000,19563520000,23987200000,23301120000,26383360000,16657920000,...,22103300,34729600,22614800,23443700,38777000,25059700,48336200,28218500,35104100,23611900


### Chaves comuns entre os dicionários


In [12]:
# Obtém as chaves de ambos os dicionários
chaves_balancos = set(dicionario_de_balancos.keys())
chaves_cotacoes = set(dicionario_de_cotacoes.keys())

# Encontra as chaves comuns
chaves_comuns = chaves_balancos.intersection(chaves_cotacoes)

# Remove as entradas dos dicionários que não estão nas chaves comuns
for chave in list(dicionario_de_balancos.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_balancos[chave]

for chave in list(dicionario_de_cotacoes.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_cotacoes[chave]

# Gera uma lista das empresas que sobraram
empresas_validadas = list(chaves_comuns)
sorted_empresas_validadas = sorted(empresas_validadas)

print("Dicionários atualizados!")
print(f"Total de empresas restantes nos balanços: {len(dicionario_de_balancos.keys())}")
print(f"Total de empresas restantes nas cotações: {len(dicionario_de_cotacoes.keys())}")
print("Lista de empresas que sobraram:")
print(sorted_empresas_validadas)

Dicionários atualizados!
Total de empresas restantes nos balanços: 466
Total de empresas restantes nas cotações: 466
Lista de empresas que sobraram:
['AALR3', 'ABCB4', 'ABEV3', 'AERI3', 'AESB3', 'AFLT3', 'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALLD3', 'ALOS3', 'ALPA3', 'ALPA4', 'ALPK3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3', 'AMBP3', 'AMER3', 'ANIM3', 'APER3', 'APTI4', 'ARML3', 'ASAI3', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AVLL3', 'AZEV3', 'AZEV4', 'AZUL4', 'AZZA3', 'B3SA3', 'BAHI3', 'BALM3', 'BALM4', 'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BDLL3', 'BDLL4', 'BEEF3', 'BEES3', 'BEES4', 'BGIP3', 'BGIP4', 'BHIA3', 'BIOM3', 'BLAU3', 'BMEB3', 'BMEB4', 'BMGB4', 'BMIN3', 'BMIN4', 'BMKS3', 'BMOB3', 'BNBR3', 'BOBR3', 'BOBR4', 'BPAC11', 'BPAC3', 'BPAC5', 'BPAN4', 'BPAR3', 'BRAP3', 'BRAP4', 'BRBI11', 'BRFS3', 'BRGE11', 'BRGE12', 'BRGE3', 'BRGE5', 'BRGE6', 'BRGE7', 'BRGE8', 'BRIT3', 'BRIV3', 'BRIV4', 'BRKM3', 'BRKM5', 'BRKM6', 'BRPR3', 'BRSR3', 'BRSR5', 'BRSR6', 'BSLI3', 'BSLI4', 'CALI3'

### Dicionarios combinados


#### Data mais antiga dos balanços e cotações de cada empresa.


In [13]:
# Função para pegar a data mais antiga
def get_oldest_date(row):
    try:
        dates = pd.to_datetime(
            row.dropna().iloc[1:], format="%d/%m/%Y", errors="coerce", dayfirst=True
        )
        if dates.empty:
            return None
        return dates.min()
    except Exception as e:
        return None


# DataFrame para armazenar os resultados
result_list = []  # Usaremos uma lista para acumular os resultados

# Iterando pelas empresas validadas
for empresa in sorted_empresas_validadas:  # Usando a lista de empresas validas
    # Caminhos dos arquivos de balanço e cotação
    balanco_file = os.path.join(BALANCOS_DEFINITIVO_FOLDER, f"{empresa}.xlsx")
    cotacao_file = os.path.join(COTACOES_FOLDER, f"{empresa}.xlsx")

    # Verifica se os arquivos existem
    if os.path.exists(balanco_file) and os.path.exists(cotacao_file):
        try:
            # Lendo a primeira linha do arquivo de balanço
            balanco_df = pd.read_excel(balanco_file, nrows=1, header=None)
            oldest_balanco_date = get_oldest_date(balanco_df.iloc[0])

            # Lendo a primeira linha do arquivo de cotação
            cotacao_df = pd.read_excel(cotacao_file, nrows=1, header=None)
            oldest_cotacao_date = get_oldest_date(cotacao_df.iloc[0])

            # Adicionando as informações à lista de resultados
            result_list.append(
                {
                    "Nome da Empresa": empresa,
                    "Data mais antiga de Balanço": oldest_balanco_date,
                    "Data mais antiga de Cotação": oldest_cotacao_date,
                }
            )
        except Exception as e:
            print(f"Erro ao processar arquivos para {empresa}: {e}")
    else:
        print(
            f"Arquivo não encontrado para {empresa}: Balanço - {balanco_file}, Cotação - {cotacao_file}"
        )

# Convertendo a lista de resultados em DataFrame
result_df = pd.DataFrame(result_list)
result_df = result_df.sort_values(by="Nome da Empresa", ascending=True).reset_index(
    drop=True
)
result_df.to_excel("datas_mais_antigas.xlsx", index=False)

# Exibindo o DataFrame final
display(result_df)

,Nome da Empresa,Data mais antiga de Balanço,Data mais antiga de Cotação
0,AALR3,2015-12-31,2016-10-31
1,ABCB4,2009-06-30,2007-09-27
2,ABEV3,2012-12-31,2000-01-06
3,AERI3,2020-09-30,2020-11-13
4,AESB3,2020-03-31,2021-03-30
...,...,...,...
461,WIZC3,2014-12-31,2015-06-08
462,WLMM3,2009-06-30,2017-07-10
463,WLMM4,2009-06-30,2000-01-04
464,YDUQ3,2009-06-30,2008-07-14


#### Igulando as datas de balanços e cotações


In [14]:
def create_date_matching_dataframe(result_df):
    def get_next_quarter_end(date):
        year = date.year
        month = date.month
        if month <= 3:
            return datetime(year, 3, 31)
        elif month <= 6:
            return datetime(year, 6, 30)
        elif month <= 9:
            return datetime(year, 9, 30)
        else:
            return datetime(year, 12, 31)

    def find_closest_previous_date(target_date, available_dates):
        available_dates = pd.to_datetime(available_dates)
        closest_date = available_dates[available_dates <= target_date].max()
        return closest_date

    final_data = []

    for _, row in result_df.iterrows():
        empresa = row["Nome da Empresa"]
        balanco_file = f"{BALANCOS_DEFINITIVO_FOLDER}/{empresa}.xlsx"
        cotacao_file = f"{COTACOES_FOLDER}/{empresa}.xlsx"

        balanco_df = pd.read_excel(balanco_file, header=None)
        cotacao_df = pd.read_excel(cotacao_file, header=None)

        balanco_dates = pd.to_datetime(
            balanco_df.iloc[0, 1:].dropna(), format="%d/%m/%Y", errors="coerce"
        )
        cotacao_dates = pd.to_datetime(
            cotacao_df.iloc[0, 1:].dropna(), format="%d/%m/%Y", errors="coerce"
        )

        start_date = max(balanco_dates.min(), cotacao_dates.min())
        end_date = min(balanco_dates.max(), cotacao_dates.max())

        current_date = start_date
        company_data = [empresa]

        while current_date <= end_date:
            if current_date in balanco_dates.values:
                balanco_date = current_date
            else:
                balanco_date = get_next_quarter_end(current_date)

            cotacao_date = find_closest_previous_date(balanco_date, cotacao_dates)

            if pd.notnull(balanco_date) and pd.notnull(cotacao_date):
                company_data.extend(
                    [
                        balanco_date.strftime("%d/%m/%Y"),
                        cotacao_date.strftime("%d/%m/%Y"),
                    ]
                )

            current_date = get_next_quarter_end(balanco_date) + timedelta(days=1)

        final_data.append(company_data)

    max_len = max(len(row) for row in final_data)
    for row in final_data:
        row.extend([""] * (max_len - len(row)))

    columns = ["Nome da Empresa"]
    for i in range(1, max_len, 2):
        columns.extend([f"Data de Balanço {i//2 + 1}", f"Data de Cotação {i//2 + 1}"])

    df = pd.DataFrame(final_data, columns=columns)
    return df


# Uso da função
date_matching_df = create_date_matching_dataframe(result_df)
date_matching_df.to_excel("datas_correspondentes.xlsx", index=False)
display(date_matching_df)

,Nome da Empresa,Data de Balanço 1,Data de Cotação 1,Data de Balanço 2,Data de Cotação 2,Data de Balanço 3,Data de Cotação 3,Data de Balanço 4,Data de Cotação 4,Data de Balanço 5,...,Data de Balanço 90,Data de Cotação 90,Data de Balanço 91,Data de Cotação 91,Data de Balanço 92,Data de Cotação 92,Data de Balanço 93,Data de Cotação 93,Data de Balanço 94,Data de Cotação 94
0,AALR3,31/12/2016,29/12/2016,31/03/2017,31/03/2017,30/06/2017,30/06/2017,30/09/2017,29/09/2017,31/12/2017,...,,,,,,,,,,
1,ABCB4,30/06/2009,30/06/2009,30/09/2009,30/09/2009,31/12/2009,30/12/2009,31/03/2010,31/03/2010,30/06/2010,...,,,,,,,,,,
2,ABEV3,31/12/2012,28/12/2012,31/03/2013,28/03/2013,30/06/2013,28/06/2013,30/09/2013,30/09/2013,31/12/2013,...,,,,,,,,,,
3,AERI3,31/12/2020,30/12/2020,31/03/2021,31/03/2021,30/06/2021,30/06/2021,30/09/2021,30/09/2021,31/12/2021,...,,,,,,,,,,
4,AESB3,31/03/2021,31/03/2021,30/06/2021,30/06/2021,30/09/2021,30/09/2021,31/12/2021,30/12/2021,31/03/2022,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,WIZC3,30/06/2015,30/06/2015,30/09/2015,30/09/2015,31/12/2015,30/12/2015,31/03/2016,31/03/2016,30/06/2016,...,,,,,,,,,,
462,WLMM3,30/09/2017,29/09/2017,31/12/2017,29/12/2017,31/03/2018,29/03/2018,30/06/2018,29/06/2018,30/09/2018,...,,,,,,,,,,
463,WLMM4,30/06/2009,30/06/2009,30/09/2009,30/09/2009,31/12/2009,30/12/2009,31/03/2010,31/03/2010,30/06/2010,...,,,,,,,,,,
464,YDUQ3,30/06/2009,30/06/2009,30/09/2009,30/09/2009,31/12/2009,30/12/2009,31/03/2010,31/03/2010,30/06/2010,...,,,,,,,,,,


#### Dicionario Combinado


In [15]:
# Dicionário para armazenar os DataFrames combinados
dicionario_combinado = {}

# Percorre cada linha do date_matching_df para cada empresa
for _, row in date_matching_df.iterrows():
    empresa = row["Nome da Empresa"]

    print(f"\nProcessando empresa: {empresa}")

    # Pegando as datas de balanço e cotação correspondentes
    datas_balanco = row[1::2]  # Colunas ímpares são as datas de balanço
    datas_cotacao = row[2::2]  # Colunas pares são as datas de cotação

    # Verifica se a empresa existe nos dois dicionários
    if empresa in dicionario_de_balancos and empresa in dicionario_de_cotacoes:
        # Pega o DataFrame de balanço e cotação para a empresa
        balanco_df = dicionario_de_balancos[empresa]
        cotacao_df = dicionario_de_cotacoes[empresa]

        # Inicializa um DataFrame vazio para armazenar a combinação de balanço e cotação
        balanco_com_cotacao = pd.DataFrame()

        # Itera sobre as datas correspondentes
        for i, (data_balanco, data_cotacao) in enumerate(
            zip(datas_balanco, datas_cotacao)
        ):

            # Se a data for vazia, interrompe o loop e passa para a próxima empresa
            if pd.isna(data_balanco) or pd.isna(data_cotacao):
                print(
                    f"Data vazia encontrada para {empresa}, encerrando processamento desta empresa."
                )
                break

            # Verifica se as datas de balanço e cotação estão presentes no DataFrame
            if (
                data_balanco in balanco_df.columns
                and data_cotacao in cotacao_df.columns
            ):
                # Pega a linha correspondente à data de balanço
                balanco_linha = balanco_df.loc[:, data_balanco]

                # Pega a linha correspondente à data de cotação
                cotacao_linha = cotacao_df.loc[:, data_cotacao]

                # Combina a linha de balanço com a de cotação, adicionando os indicadores de cotação ao final
                linha_combinada = pd.concat([balanco_linha, cotacao_linha])

                # Adiciona a linha combinada ao DataFrame final com a data de balanço como coluna
                balanco_com_cotacao[data_balanco] = linha_combinada

        # Verifica se o DataFrame final para a empresa não está vazio
        if not balanco_com_cotacao.empty:
            # Armazena o DataFrame final no dicionário combinado com o nome da empresa como chave
            dicionario_combinado[empresa] = balanco_com_cotacao
            print(f"Processamento finalizado para {empresa}.")
        else:
            print(
                f"Nenhuma data válida encontrada para {empresa}. Empresa não será adicionada ao dicionário."
            )
    else:
        print(
            f"Empresa {empresa} não encontrada nos dicionários de balanço ou cotação."
        )

# Impressão das empresas adicionadas ao dicionário
print("\nEmpresas adicionadas ao dicionário combinado:")
for key in dicionario_combinado.keys():
    print(f"- {key}")

# Número total de empresas processadas
print(f"\nNúmero total de empresas processadas: {len(dicionario_combinado.keys())}")

# Verifica se alguma empresa não foi adicionada
empresas_nao_adicionadas = [
    empresa
    for empresa in date_matching_df["Nome da Empresa"]
    if empresa not in dicionario_combinado.keys()
]
if empresas_nao_adicionadas:
    print("\nEmpresas que não foram adicionadas ao dicionário (sem datas válidas):")
    for empresa in empresas_nao_adicionadas:
        print(f"- {empresa}")
else:
    print("\nTodas as empresas foram adicionadas ao dicionário.")


Processando empresa: AALR3
Processamento finalizado para AALR3.

Processando empresa: ABCB4
Processamento finalizado para ABCB4.

Processando empresa: ABEV3
Processamento finalizado para ABEV3.

Processando empresa: AERI3
Processamento finalizado para AERI3.

Processando empresa: AESB3
Processamento finalizado para AESB3.

Processando empresa: AFLT3
Processamento finalizado para AFLT3.

Processando empresa: AGRO3
Processamento finalizado para AGRO3.

Processando empresa: AGXY3
Processamento finalizado para AGXY3.

Processando empresa: AHEB3
Processamento finalizado para AHEB3.

Processando empresa: AHEB5
Processamento finalizado para AHEB5.

Processando empresa: AHEB6
Processamento finalizado para AHEB6.

Processando empresa: ALLD3
Processamento finalizado para ALLD3.

Processando empresa: ALOS3
Processamento finalizado para ALOS3.

Processando empresa: ALPA3
Processamento finalizado para ALPA3.

Processando empresa: ALPA4
Processamento finalizado para ALPA4.

Processando empresa: ALP

In [16]:
# Visualizar o primeiro resultado do dicionário combinado para verificar
dicionario_combinado["AALR3"]

,31/12/2016,31/03/2017,30/06/2017,30/09/2017,31/12/2017,31/03/2018,30/06/2018,30/09/2018,31/12/2018,31/03/2019,...,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024,30/06/2024
AALR3,,,,,,,,,,,,,,,,,,,,,
Ativo Total,2166311.936,2218670.08,2270885.12,2308239.104,2284985.088,2274482.944,2263314.944,2258248.96,2209425.92,2429751.04,...,2537389.056,2573832.96,2631220.992,2615453.952,2571685.12,2520962.048,2748189.952,2669810.944,2573677.056,2692409.088
Ativo Circulante,494428.992,384152.992,409799.008,430387.008,401553.984,399984.992,409183.008,423904.992,399736,401603.008,...,456177.984,457827.008,508798.016,484284,433144,386719.008,620451.968,524747.008,441300.992,563769.024
Caixa e Equivalentes de Caixa,159332.992,53505,51155,50806,91597,68523,63455,46970,73165,64266,...,81625,240190,247828.992,218744,154614,95520,294614.016,218595.008,97788,254478
Aplicações Financeiras,37811,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Contas a Receber,233260,249166,263584,282444,219178,235863.008,243296.992,264752.992,228960.992,239566,...,275668.992,133735,171106,186219.008,189298,206091.008,236254,202160,234380,186158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
High,14.8,15.63,17.540001,17.040001,14.8,16.1,13.14,11.8,13.7,15.01,...,18.469999,19.57,21.120001,21.6,22.6,23.82,9.77,10.88,9.17,10.48
Low,14,14.87,17.280001,16.799999,14.8,15.32,12.82,11.5,13.13,14.52,...,17.83,18.77,20.870001,21.16,22.5,23.67,9.16,10.33,8.83,9.97
Close,14.65,15.63,17.540001,17.040001,14.8,16.1,13.08,11.68,13.31,14.52,...,17.950001,19.51,20.98,21.6,22.530001,23.799999,9.77,10.4,9.17,10.39


#### Dicionario Combinando para Excel


In [17]:
def salvar_dicionario_em_xlsx(dicionario_combinado, pasta_destino):
    # Cria a pasta de destino, se ela não existir
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    # Contador para o número de empresas salvas
    total_empresas_salvas = 0

    # Percorre cada empresa no dicionário combinado
    for empresa, df in dicionario_combinado.items():
        # Define o caminho completo do arquivo .xlsx
        caminho_arquivo = os.path.join(pasta_destino, f"{empresa}.xlsx")

        try:
            # Salva o DataFrame em formato Excel
            df.to_excel(caminho_arquivo, index=True)

            # Incrementa o contador de sucesso
            total_empresas_salvas += 1

            # Mensagem de sucesso para cada arquivo salvo
            print(f"Arquivo {caminho_arquivo} salvo com sucesso.")
        except Exception as e:
            # Mensagem de erro caso ocorra uma falha
            print(f"Falha ao salvar {caminho_arquivo}: {e}")

    # Exibe o total de empresas salvas
    print(f"\nTotal de empresas salvas como arquivos Excel: {total_empresas_salvas}")


# Exemplo de uso da função
pasta_destino = BALANCOS_COTACOES_COMBINADOS_FOLDER
salvar_dicionario_em_xlsx(dicionario_combinado, pasta_destino)

Arquivo balancos_cotacoes_combinados\AALR3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ABCB4.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ABEV3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AERI3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AESB3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AFLT3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AGRO3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AGXY3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AHEB3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AHEB5.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\AHEB6.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ALLD3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ALOS3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ALPA3.xlsx salvo com sucesso.
Arquivo balancos_cotacoes_combinados\ALPA4.xlsx salvo com suce

## Graham e PEG


In [2]:
def obter_numero_acoes_em_circulacao(empresa):
    """Obtém o número de ações em circulação de uma empresa no Yahoo Finance."""
    ticker = yf.Ticker(empresa + ".SA")
    numero_acoes = ticker.info.get("sharesOutstanding")
    return numero_acoes


def calcular_graham(lpa, vpa):
    """Calcula o valor intrínseco de uma ação usando a fórmula de Benjamin Graham."""
    try:
        valor = 22.5 * lpa * vpa
        return np.sqrt(valor) if valor >= 0 else 0
    except Exception as e:
        print(f"Erro ao calcular o valor intrínseco de Graham: {e}")
        return None


# Dicionários para armazenar os dados trimestrais e anuais
dicionario_trimestral = {}
dicionario_anual = {}
empresas_removidas = []
empresas_processadas = []
contador_de_empresas = 0
empresas_com_problemas = []

# Iterar pelas empresas no dicionário combinado
for empresa in list(dicionario_combinado.keys()):
    arquivo = f"{empresa}.xlsx"
    caminho_arquivo = os.path.join(BALANCOS_COTACOES_COMBINADOS_FOLDER, arquivo)

    try:
        # Lê a planilha e ajusta o índice e as colunas corretamente
        df = pd.read_excel(caminho_arquivo, header=None)
        nome = df.iloc[0, 0]  # Nome da empresa na célula [0,0]
        print(f"Processando empresa {nome}...")

        # Configurar cabeçalho e índice
        df.columns = df.iloc[0]
        df = df[1:]  # Remove a linha do cabeçalho
        df = df.set_index(df.columns[0])

        # Obtém o número de ações em circulação
        numero_acoes = obter_numero_acoes_em_circulacao(empresa)
        if numero_acoes is None or numero_acoes == 0:
            print(
                f"Empresa {empresa} removida por falta de dados de ações em circulação."
            )
            empresas_removidas.append(empresa)
            dicionario_combinado.pop(empresa)
            continue

        # Dicionário temporário para armazenar dados trimestrais de cada coluna
        dados_trimestrais = {}
        dados_anuais = {}

        # Variáveis de controle para cálculos anuais
        lucro_liquido_acumulado = 0
        ano_atual = None

        # Calcula os indicadores financeiros para cada data
        for coluna in df.columns[1:]:
            try:
                data = pd.to_datetime(coluna, dayfirst=True)
                ano = data.year

                # Extrair valores financeiros
                patrimonio_liquido = df.loc["Patrimônio Líquido", coluna] * 1000
                lucro_liquido = df.loc["Lucro/Prejuízo do Período", coluna] * 1000
                preco_atual = df.loc["Adj Close", coluna]

                # Cálculos dos indicadores
                lpa = lucro_liquido / numero_acoes
                vpa = patrimonio_liquido / numero_acoes
                graham = calcular_graham(lpa, vpa)

                # Adicionar dados trimestrais no formato solicitado
                if coluna not in dados_trimestrais:
                    dados_trimestrais[coluna] = {
                        "LPA": lpa,
                        "VPA": vpa,
                        "Preço": preco_atual,
                        "Graham": graham,
                        "Lucro Líquido": lucro_liquido,
                        "Patrimônio Líquido": patrimonio_liquido,
                    }

                # Acumula o lucro líquido e guarda o patrimônio líquido do último trimestre
                if ano_atual is None:
                    ano_atual = ano

                if ano != ano_atual:
                    # Quando o ano muda, armazena os dados anuais do ano anterior
                    lpa_anual = lucro_liquido_acumulado / numero_acoes
                    vpa_anual = patrimonio_liquido_ultimo / numero_acoes
                    dados_anuais[ano_atual] = {
                        "Lucro Líquido Anual": lucro_liquido_acumulado,
                        "Patrimônio Líquido Anual": patrimonio_liquido_ultimo,
                        "LPA Anual": lpa_anual,
                        "VPA Anual": vpa_anual,
                        "Graham": graham,
                    }

                    # Reinicializa variáveis para o novo ano
                    lucro_liquido_acumulado = 0
                    ano_atual = ano

                # Acumular lucro líquido e atualizar patrimônio líquido do último trimestre
                lucro_liquido_acumulado += lucro_liquido
                patrimonio_liquido_ultimo = patrimonio_liquido

            except Exception as e:
                print(
                    f"Erro ao calcular indicadores para {empresa} na data {coluna}: {e}"
                )
                empresas_com_problemas.append(empresa)
                continue

        # Finalizar o ano atual
        if ano_atual:
            lpa_anual = lucro_liquido_acumulado / numero_acoes
            vpa_anual = patrimonio_liquido_ultimo / numero_acoes
            dados_anuais[ano_atual] = {
                "Lucro Líquido Anual": lucro_liquido_acumulado,
                "Patrimônio Líquido Anual": patrimonio_liquido_ultimo,
                "LPA Anual": lpa_anual,
                "VPA Anual": vpa_anual,
                "Graham": graham,
            }

        # Converte os dicionários em DataFrames e armazena no formato transposto solicitado
        dicionario_trimestral[empresa] = pd.DataFrame(dados_trimestrais)
        dicionario_anual[empresa] = pd.DataFrame(dados_anuais)
        empresas_processadas.append(empresa)
        contador_de_empresas += 1

    except Exception as e:
        print(f"Erro ao processar a empresa {empresa}: {e}")
        empresas_com_problemas.append(empresa)

# Impressão do resumo final
print("\nEmpresas processadas:")
for empresa in dicionario_trimestral.keys():
    print(f"- {empresa}")

print(f"\nTotal de empresas processadas: {len(dicionario_trimestral.keys())}")
print(f"Empresas removidas: {empresas_removidas}")
print(f"Empresas com problemas: {empresas_com_problemas}")

NameError: name 'dicionario_combinado' is not defined

In [22]:
# Exemplo de visualização dos indicadores para uma empresa
dicionario_trimestral["AALR3"]

,31/03/2017,30/06/2017,30/09/2017,31/12/2017,31/03/2018,30/06/2018,30/09/2018,31/12/2018,31/03/2019,30/06/2019,...,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024,30/06/2024
LPA,3.799604e-02,2.596550e-02,6.691008e-02,-8.718864e-02,4.664464e-02,3.460096e-02,7.369368e-02,1.083078e-01,6.492031e-02,6.301591e-02,...,-1.086624e-01,-8.205333e-02,-4.933247e-01,-8.119635e-01,-2.375836e-01,-3.387138e-01,-3.399155e-01,-5.802376e-01,-5.171034e-01,2.843465e-03
VPA,7.768011e+00,8.000735e+00,8.079807e+00,8.008064e+00,8.078763e+00,8.120817e+00,8.202792e+00,8.254230e+00,8.305544e+00,8.355617e+00,...,7.613643e+00,7.521509e+00,7.043348e+00,6.236552e+00,5.955824e+00,5.682845e+00,6.666139e+00,6.024836e+00,6.364574e+00,7.151643e+00
Preço,1.547291e+01,1.736372e+01,1.686874e+01,1.465125e+01,1.593819e+01,1.294854e+01,1.156261e+01,1.317623e+01,1.437407e+01,1.430158e+01,...,1.795000e+01,1.951000e+01,2.098000e+01,2.160000e+01,2.253000e+01,2.380000e+01,9.770000e+00,1.040000e+01,9.170000e+00,1.039000e+01
Graham,2.577006e+00,2.161994e+00,3.487687e+00,0.000000e+00,2.911820e+00,2.514405e+00,3.687969e+00,4.484968e+00,3.483097e+00,3.441958e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.764226e-01
Lucro Líquido,5.786000e+06,3.954000e+06,1.018900e+07,-1.327700e+07,7.103000e+06,5.269000e+06,1.122200e+07,1.649300e+07,9.886000e+06,9.596000e+06,...,-1.654700e+07,-1.249500e+07,-7.512300e+07,-1.236450e+08,-3.617900e+07,-5.157900e+07,-5.176200e+07,-8.835800e+07,-7.874400e+07,4.330000e+05
Patrimônio Líquido,1.182905e+09,1.218344e+09,1.230385e+09,1.219460e+09,1.230226e+09,1.236630e+09,1.249113e+09,1.256946e+09,1.264760e+09,1.272385e+09,...,1.159398e+09,1.145368e+09,1.072554e+09,9.496960e+08,9.069470e+08,8.653780e+08,1.015113e+09,9.174560e+08,9.691910e+08,1.089045e+09


In [23]:
dicionario_anual["AALR3"]

,2017,2018,2019,2020,2021,2022,2023,2024
Lucro Líquido Anual,6.652000e+06,4.008700e+07,4.130000e+07,-9.741600e+07,-5.625000e+06,-2.278100e+08,-2.278780e+08,-7.831100e+07
Patrimônio Líquido Anual,1.219460e+09,1.256946e+09,1.282190e+09,1.190537e+09,1.175292e+09,9.496960e+08,9.174560e+08,1.089045e+09
LPA Anual,4.368297e-02,2.632471e-01,2.712127e-01,-6.397205e-01,-3.693877e-02,-1.496004e+00,-1.496451e+00,-5.142600e-01
VPA Anual,8.008064e+00,8.254230e+00,8.420005e+00,7.818129e+00,7.718017e+00,6.236552e+00,6.024836e+00,7.151643e+00
Graham,2.911820e+00,3.483097e+00,0.000000e+00,3.433954e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.764226e-01
Taxa de Crescimento,1.496716e-01,5.928275e+00,6.137919e+00,NaN,NaN,NaN,NaN,NaN
PEG,2.437740e+03,9.210598e+00,5.362040e+00,NaN,NaN,NaN,NaN,NaN
